<a href="https://colab.research.google.com/github/NinaMwangi/Databases_Formative/blob/main/PLG_4_FETCH_DATA_FOR_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Netflix Dataset



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 52.9 MB/s eta 0:00:00


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pandas as pd
import joblib
import requests
import numpy as np
from tensorflow.keras.models import load_model
import mysql.connector
import numpy as np
from tensorflow.keras.models import load_model

# Loading train and test dataset

In [6]:
# Load dataset
data = pd.read_csv('/content/drive/MyDrive/ml_file (1).csv')

In [7]:
# prompt: print column in the data

data.columns

Index(['movie_id', 'title', 'movie_type', 'netflix_exclusive', 'release_date'], dtype='object')

In [11]:
# Selection fearures and target for prediction
# Assuming 'netflix exclusive' as the target varaible for prediction
features = ["title", "movie_type", "netflix_exclusive", "release_date",]

target = "netflix_exclusive"

In [12]:
# Preprocess data: Label encoding categorical variables
label_encoders = {}
for feature in features:
    if feature in data.columns:  # Check if the feature exists in the DataFrame
        le = LabelEncoder()
        data[feature] = le.fit_transform(data[feature])
        label_encoders[feature] = le
    else:
        print(f"Warning: Feature '{feature}' not found in the DataFrame. Skipping.")

In [13]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

In [14]:
# Train model (Random Forest)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [15]:
# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 1.00


In [16]:
# Save the model and label encoders
joblib.dump(model, 'netflix_exclusive_model.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')

['label_encoders.pkl']

In [17]:
# Load model and encoder
model = joblib.load('netflix_exclusive_model.pkl')
label_encoders = joblib.load('label_encoders.pkl')

# Load the Pre-trained Model

In [18]:
print(data)


     movie_id  title  movie_type  netflix_exclusive  release_date
0           1     16           4                  1           414
1           2    580           3                  1            95
2           3    349           3                  1            33
3           4     29           3                  0            70
4           5     79           1                  0            96
..        ...    ...         ...                ...           ...
639       641     22           1                  1           411
640       642    566           1                  1           412
641       643    357           3                  1           413
642       644    407           1                  0            47
643       645    406           1                  0             4

[644 rows x 5 columns]


In [19]:
# Prepare data for prediction
# Convert latest_movies dictionary to DataFrame for processing
df_latest = data
df_latest.head()

,movie_id,title,movie_type,netflix_exclusive,release_date
0,1,16,4,1,414
1,2,580,3,1,95
2,3,349,3,1,33
3,4,29,3,0,70
4,5,79,1,0,96


In [20]:
features = ["title", "movie_type", "netflix_exclusive", "release_date",]

In [21]:
# Get the actual column names from the DataFrame
available_features = df_latest.columns.tolist()

# Print the available features for debugging
print(f"Available features in df_latest: {available_features}")


Available features in df_latest: ['movie_id', 'title', 'movie_type', 'netflix_exclusive', 'release_date']


In [22]:
print(df_latest.columns)

Index(['movie_id', 'title', 'movie_type', 'netflix_exclusive', 'release_date'], dtype='object')


In [23]:
# List of features used during training
trained_features = ['title', 'movie_type', 'netflix_exclusive', 'release_date']

In [24]:
x_latest = df_latest[trained_features]

# Make prediction

In [33]:
# Find common features
common_features = list(set(trained_features) & set(available_features))

# Ensure features are in the same order as during training
common_features = [feature for feature in trained_features if feature in common_features]

# Select only the common features for prediction
x_latest = df_latest[common_features]

# Predict using the loaded model
predicted_exclusive = model.predict(x_latest)  # Assign prediction output to 'predicted_exclusive'

# Now apply inverse_transform (only if 'netflix_exclusive' is in common_features)
if 'netflix_exclusive' in common_features:
    predicted_label = label_encoders["netflix_exclusive"].inverse_transform(predicted_exclusive)
    print(predicted_label)
else:
    print("Netflix exclusive feature not found in the data.")

[1 1 1 0 0 1 1 1 1 0 1 1 1 0 0 0 1 1 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 0 0 1 1
 1 1 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1
 1 0 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 0 0 0 1 1 1 1 1 1 1 0 0 1 0
 1 1 0 0 0 0 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 0 1 1 0 1 1 0 0 1 1 1 0 1 0 1
 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 0 0 1 0 0 0 0 0 1 1 1 1 0 0 1
 1 1 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1 1 0 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 1 0
 1 1 0 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 1 1 0 1 1
 0 1 1 0 1 1 1 1 0 1 1 0 1 1 0 0 0 1 0 1 1 0 0 0 1 1 1 1 0 1 1 0 1 1 1 1 1
 1 0 0 1 0 0 0 0 0 1 1 1 1 1 1 0 0 1 1 1 0 1 0 0 0 1 1 0 0 1 1 1 1 1 0 1 0
 1 0 0 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 0 1
 0 0 1 1 1 1 0 1 0 0 1 0 1 0 1 1 1 1 0 0 0 1 0 1 1 1 0 0 0 1 1 1 0 0 1 1 1
 0 1 1 1 1 0 0 0 0 1 1 1 0 0 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 1 1 0 1 1 1 0
 1 0 0 1 1 1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 0 1 0 0 1
 1 1 1 1 0 1 1 1 1 1 1 1 

In [34]:
def tranform(predictions):
  trans = []
  for i in predictions:
    if i== "0":
      val = "no"
    else:
        val= "yes"
    trans.append(val) # Intended to be inside the 'else block'
  return trans

In [42]:
trans = tranform(predicted_label)

print(trans)

['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes'